<a href="https://colab.research.google.com/github/JulioEI/TBCG_Group12/blob/main/testing_CNN_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TESTING PIPELINE


In [1]:
import os
import tensorflow as tf
import sys
!git clone https://github.com/JulioEI/TBCG_Group12.git
sys.path.insert(0,'/content/TBCG_Group12/code')
import utils as ut
import bcg_auxiliary as bcg

Cloning into 'TBCG_Group12'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 194 (delta 79), reused 164 (delta 54), pack-reused 0
Receiving objects: 100% (194/194), 1.20 MiB | 3.86 MiB/s, done.
Resolving deltas: 100% (79/79), done.


Define general parameters

In [2]:
fs=1250 #sampling frequency after downsampling
window_seconds = 0.05 #window length in seconds
overlapping = 0.7 #window overlapping

Get testing data from figshare

In [3]:
#TEST DATA
!wget https://figshare.com/ndownloader/articles/16856182/versions/1 -O zip_Test1
!unzip /content/zip_Test1 -d /content/Test1
!rm zip_Test1
!wget https://figshare.com/ndownloader/articles/16856248/versions/1 -O zip_Test2
!unzip /content/zip_Test2 -d /content/Test2
!rm zip_Test2

--2021-11-15 13:30:09--  https://figshare.com/ndownloader/articles/16856182/versions/1
Resolving figshare.com (figshare.com)... 34.251.171.26, 52.19.36.181, 2a05:d018:1f4:d003:51ae:4c8f:af50:efaa, ...
Connecting to figshare.com (figshare.com)|34.251.171.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 727991274 (694M) [application/zip]
Saving to: ‘zip_Test1’

zip_Test1           100%[===================>] 694.27M  20.0MB/s    in 37s     

2021-11-15 13:30:47 (18.8 MB/s) - ‘zip_Test1’ saved [727991274/727991274]

Archive:  /content/zip_Test1
 extracting: /content/Test1/ephys_shank4.dat  
 extracting: /content/Test1/info.mat  
 extracting: /content/Test1/mapsCh.csv  
--2021-11-15 13:30:53--  https://figshare.com/ndownloader/articles/16856248/versions/1
Resolving figshare.com (figshare.com)... 52.19.36.181, 34.251.171.26, 2a05:d018:1f4:d003:51ae:4c8f:af50:efaa, ...
Connecting to figshare.com (figshare.com)|52.19.36.181|:443... connected.
HTTP request sent, awai

Load downloaded data and preprocess it

In [6]:
datapath = "/content/Test1"
data_test1, x_test1, indx_map_test1 = ut.load_test_data_pipeline(datapath, desired_fs=fs, 
                             window_seconds = window_seconds, overlapping = overlapping, zscore=True, binary = False)

datapath = "/content/Test2"
data_test2, x_test2, indx_map_test2 = ut.load_test_data_pipeline(datapath, desired_fs=fs, 
                             window_seconds = window_seconds, overlapping = overlapping, zscore=True, binary = False)

Load model from github repo

In [7]:
model = tf.keras.models.load_model("/content/TBCG_Group12/code/model/model_prob_vf.h5")

Get predictions

In [8]:
y_test1 = model.predict(x_test1)
y_test2 = model.predict(x_test2)

events_prediction_test1 = ut.get_ripple_times_from_CNN_output(y_test1, indx_map_test1,
                                              th_zero = 5e-1,th_dur = 0.01, verbose = False)
events_prediction_test2 = ut.get_ripple_times_from_CNN_output(y_test2, indx_map_test2,
                                              th_zero = 5e-1,th_dur = 0.01, verbose = False)

Save results into csv

In [ ]:
!mkdir '/content/results'
bcg.write_results("/content/results/", 'test1_results',12, events_prediction_test1)
bcg.write_results("/content/results/", 'test2_results',12, events_prediction_test2)